**Source**: [Neural Networks and Deep Learning - Chapter 1](http://neuralnetworksanddeeplearning.com/chap1.html)

### 🧠 Sigmoid Neuron

A **sigmoid neuron** takes inputs in the range [0, 1] and outputs a value between 0 and 1, using the sigmoid activation function.

The sigmoid function is defined as:

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

---

### 🧱 Neural Network Layers

- **Input Layer**: Contains *input neurons*
- **Hidden (Middle) Layer**: Contains *hidden neurons*
- **Output Layer**: Contains *output neurons*

---

### 🔻 Gradient Descent

Let:
- $x$ be the training input  
- $y = y(x)$ be the corresponding desired output  
- $a$ be the actual output of the network  

The **cost function** is defined as:

$$
C(w, b) = \frac{1}{2n} \sum_x \| y(x) - a \|^2
$$

Suppose we simplify notation by writing $(w, b) \rightarrow (v_1, v_2)$.  
Then the change in cost is approximately:

$$
\Delta C \approx \frac{\partial C}{\partial v_1} \Delta v_1 + \frac{\partial C}{\partial v_2} \Delta v_2
$$

We want to choose $\Delta v_1$ and $\Delta v_2$ such that $\Delta C < 0$ (i.e., the cost decreases).

Define:
- $\Delta v \equiv \begin{pmatrix} \Delta v_1 \\ \Delta v_2 \end{pmatrix}$
- $\nabla C \equiv \begin{pmatrix} \frac{\partial C}{\partial v_1} \\ \frac{\partial C}{\partial v_2} \end{pmatrix}$

Then:

$$
\Delta C \approx \nabla C \cdot \Delta v
$$

To ensure the cost decreases, we choose:

$$
\Delta v = -\eta \nabla C
$$

where $\eta$ is a small positive number called the *learning rate*. Substituting, we get:

$$
\Delta C \approx -\eta \nabla C \cdot \nabla C = -\eta \| \nabla C \|^2
$$

Since $\| \nabla C \|^2 \geq 0$, this guarantees:

$$
\Delta C \leq 0
$$

So, the cost function $C$ always decreases (or remains the same), and the update rule becomes:

$$
v \rightarrow v' = v - \eta \nabla C
$$

This can be extended to functions with any number of variables.


Excercise 1: The goal is to minimize the cost $C$ in such a way that the cost $C$ goes down as much as possible. Let's the limit the size of the change to a fixed value, $ \|\Delta v\| = \epsilon$. As a first order of approximation $\Delta C \approx  \nabla C \cdot \Delta v$. Now the objective is to choose a vector $\Delta v$ of fixed length $\epsilon$ that minimizes $\nabla C \cdot \Delta v$. The dot product of $\nabla C \cdot \Delta v$ is $\|\nabla C\|\cdot \|\Delta v\| \cos(\theta)$. To minimize this, $\cos(\theta) = -1$ since $ \|\Delta v\| = \epsilon$, the smallest value of the dot product is $-\|\nabla C\|\cdot \epsilon$, hence $\eta = \epsilon / \|\nabla C\|$.

Excecise 2: For $1D$, the "gradient" is just the slope of the line.

### 🔁 Gradient Descent (Component-wise Update)

When using gradient descent to minimize a cost function $C$, we update each **weight** $w_k$ and **bias** $b_l$ individually using the gradients of the cost function:

$$
w_k \rightarrow w_k' = w_k - \eta \frac{\partial C}{\partial w_k}
$$

$$
b_l \rightarrow b_l' = b_l - \eta \frac{\partial C}{\partial b_l}
$$

where:
- $\eta$ is the **learning rate**,
- $\frac{\partial C}{\partial w_k}$ and $\frac{\partial C}{\partial b_l}$ are the **partial derivatives** of the cost function with respect to each parameter.

---

### 🎲 Stochastic Gradient Descent (SGD)

Instead of computing the full gradient $\nabla C$ over the entire dataset (which can be computationally expensive), **Stochastic Gradient Descent** estimates it using a small, randomly selected subset (mini-batch) of the training data.

We approximate the true gradient:

$$
\nabla C \approx \frac{1}{m} \sum_{j=1}^{m} \nabla C_{x_j}
$$

where:
- $m$ is the **mini-batch size**,
- $x_j$ is the $j$-th example in the mini-batch,
- $\nabla C_{x_j}$ is the gradient computed for the individual training example $x_j$.

This is an approximation to the full batch gradient:

$$
\nabla C = \frac{1}{n} \sum_{x} \nabla C_x
$$

---

### 🧮 SGD Component-wise Update Rules

Using the estimated gradient from the mini-batch, we update the weights and biases as:

$$
w_k \rightarrow w_k' = w_k - \frac{\eta}{m} \sum_{j=1}^{m} \frac{\partial C_{x_j}}{\partial w_k}
$$

$$
b_l \rightarrow b_l' = b_l - \frac{\eta}{m} \sum_{j=1}^{m} \frac{\partial C_{x_j}}{\partial b_l}
$$

This speeds up training and helps the model generalize better due to the noise introduced by random sampling.


In [1]:
import numpy as np

In [4]:
class Network(object):
    def __init__(self, sizes):
        """Initialize the network with a list of sizes."""
        self.num_layers = len(sizes) # number of layers in the network
        self.sizes = sizes # number of neurons in each layer
        self.biases = [np.random.randn(cur, 1) for cur in sizes[1:]] # biases for each layer (except the input layer)
        self.weights = [np.random.randn(prev, cur) for prev, cur in zip(sizes[:-1], sizes[1:])] # weights for each layer (except the input layer)
    def feedforward(self, a):
        """Return the output of the network given input a."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """Train the network using mini batch stochastic gradient descent (SGD). The 
        training_data is a list of tuples (x, y) where x is the input and y is the expected output.
        The test_data is a list of tuples (x, y) for evaluating the performance of the network. Other parameters
        are epochs (number of iterations), mini_batch_size (size of each mini batch), and eta (learning rate)."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            np.random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print(f"Epoch {j}: {self.evaluate(test_data)} / {n_test}")
            else:
                print(f"Epoch {j} complete")

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying gradient descent using a single mini batch."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w - (eta/len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta/len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)]
    
    def backprop(self, x, y):
        """Return a tuple (nabla_b, nabla_w) representing the gradient of the cost function
        with respect to the biases and weights of the network."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x]  
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural network outputs the correct result."""
        test_results = [(np.argmax(self.feedforward(x)), np.argmax(y)) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives of the cost function."""
        return output_activations - y


def sigmoid_prime(z):
    """Return the derivative of the sigmoid function."""
    return sigmoid(z) * (1 - sigmoid(z))
    
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

In [3]:
"""
mnist_loader
~~~~~~~~~~~~

A library to load the MNIST image data.  For details of the data
structures that are returned, see the doc strings for ``load_data``
and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the
function usually called by our neural network code.
"""

#### Libraries
# Standard library
import pickle
import gzip

# Third-party libraries
import numpy as np

def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.

    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.

    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.

    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.

    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('../data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f)
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.

    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.

    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.

    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


In [5]:
net = Network([2, 3, 1]) # Example: a network with 2 input neurons, 3 hidden neurons, and 1 output neuron
net.weights

[array([[-1.54174811,  0.44301886],
        [-1.17677569, -2.97927487],
        [-0.33045123, -0.28177142]]),
 array([[ 0.24636485, -0.45324362, -0.58637459]])]

Backpropagation: The activation of the $j$-th neuron in layer $l$ is given by:


($a_1^{l-1}$)         *
                        .
                            .   
($a_1^{l-1}$)         * . . . *

                            .
                        .
($a_1^{l-1}$)         *       *


layer   l-1     l
$$
a_j^{(l)} = \sigma\left( \sum_k w_{jk}^{(l)} a_k^{(l-1)} + b_j^{(l)} \right)
$$